## Flow validation

This notebook has the objective of testing 

In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.0.0'

In [21]:
# create a matrix

matrix = np.zeros((5,5))
matrix[1,2] = 1
matrix[2,3] = 1
matrix = np.array([[3, 3, 3, 4, 0, 3],
       [3, 3, 0, 1, 2, 0],
       [1, 2, 4, 3, 4, 4],
       [2, 0, 2, 4, 2, 0],
       [4, 0, 1, 0, 0, 2]])

In [22]:

c = 0

h_matrix = np.zeros_like(matrix)

def _2d_rec(h_matrix, i,j):
    
    if i<0 or j<0:
        return 0, h_matrix
    
    h_matrix[i,j] = _2d_rec(h_matrix, i, j-1)[0] + _2d_rec(h_matrix, i-1, j)[0] + _2d_rec(h_matrix, i-1, j-1)[0] + matrix[i,j]
    global c
    c+=1
    return h_matrix[i,j], h_matrix

print(_2d_rec(h_matrix, h_matrix.shape[0]-1, h_matrix.shape[1]-1))
print(c)

(5054, array([[   3,    6,    9,   13,   13,   16],
       [   6,   18,   33,   56,   84,  113],
       [   7,   33,   88,  180,  324,  525],
       [   9,   49,  172,  444,  950, 1799],
       [  13,   71,  293,  909, 2303, 5054]]))
1826


In [12]:
# Dynamic programing TRICKs

c = 0

h_matrix = np.zeros((5,5)) - 1

def _2d_rec(h_matrix, i,j):
    
    if i<0 or j<0:
        return 0
    
    if h_matrix[i,j] !=- 1: # MEMOIZATION
        return h_matrix[i,j]
    
    h_matrix[i,j] = _2d_rec(h_matrix, i, j-1) + _2d_rec(h_matrix, i-1, j) + _2d_rec(h_matrix, i-1, j-1) + matrix[i,j]
    global c
    c+=1
    return h_matrix[i,j]

print(rec(h_matrix, 4,4))
print(h_matrix)
print(c)

30.0
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  4.  6.]
 [ 0.  0.  1.  6. 16.]
 [ 0.  0.  1.  8. 30.]]
25


In [11]:
# 2D sequential approach

h_matrix = np.zeros((5,5))
matrix = np.zeros((5,5))
matrix[1,2] = 1
matrix[2,3] = 1
c = 0

def _2d_sequential(hi,hj):
    for i in range(1,hi+1):
        for j in range(1,hj+1):
            h_matrix[i,j] = h_matrix[i,j-1] + h_matrix[i-1,j] + h_matrix[i-1,j-1] + matrix[i,j]
            global c
            c+=1
            
    return h_matrix[hi,hj]

print(_2d_sequential(4,4))
print(h_matrix)
print(c)

30.0
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  4.  6.]
 [ 0.  0.  1.  6. 16.]
 [ 0.  0.  1.  8. 30.]]
16


In [10]:
# sequencial single loop

h_matrix = np.zeros(5*5+1)
matrix = np.zeros((5,5))
matrix[1,2] = 1
matrix[2,3] = 1
#matrix = matrix.flatten()


def _1d_sequential(hi,hj):
    
    def _1d_to_2d(index):
        return (index//(hi+1), index%(hj+1))
    
    def _2d_to_1d(k,l):
        if k<0 or l<0:
            return (hi+1)*(hj+1)
        return k*(hi+1)+l
    
    for index in range(0,(hi+1)*(hj+1)):
        
        i,j = _1d_to_2d(index)
        h_matrix[_2d_to_1d(i,j)] = h_matrix[_2d_to_1d(i,j-1)] + h_matrix[_2d_to_1d(i-1,j)] + h_matrix[_2d_to_1d(i-1,j-1)] + matrix[i,j]
            
    return h_matrix[_2d_to_1d(hi,hj)]

print(_1d_sequential(4,4))
print(h_matrix)
print(np.reshape(h_matrix[:25], (5,5)))


30.0
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.  4.  6.  0.  0.  1.
  6. 16.  0.  0.  1.  8. 30.  0.]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  4.  6.]
 [ 0.  0.  1.  6. 16.]
 [ 0.  0.  1.  8. 30.]]


In [30]:
# sequencial single loop preaty close to the TensorFlow instructions and capabilities

h_matrix = np.zeros(5*5)
matrix = np.zeros((5,5))
matrix[1,2] = 1
matrix[2,3] = 1


def _1d_sequential(matrix):
    
    rows = matrix.shape[0]
    colums = matrix.shape[1]
    matrix = matrix.flatten()
    
    initial_state = 0
    
    def get_back(index, colums):
        if 0>=index%colums:
            return initial_state
        else:
            return h_matrix[index-1]
    
    def get_up(index, colums):
        if index<colums:
            return initial_state
        else:
            return h_matrix[index-colums]
    
    def get_diagonal(index, colums):
        if 0>=index%colums or index<colums:
            return initial_state
        else:
            return h_matrix[index-colums-1]
    
    for index in range(rows*colums):

        h_matrix[index] = get_back(index, colums) + get_up(index, colums) + get_diagonal(index, colums) + matrix[index]
            
    return h_matrix[index]

print(_1d_sequential(matrix))
print(h_matrix)
print(np.reshape(h_matrix, (5,5)))


30.0
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1.  4.  6.  0.  0.  1.
  6. 16.  0.  0.  1.  8. 30.]
[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  4.  6.]
 [ 0.  0.  1.  6. 16.]
 [ 0.  0.  1.  8. 30.]]


## TensorFlow 2.0 implementation

Now let's the tensorflow stuff begin!

(The code was build arround the tf.while_loop mentallity, so the convertion to tf.1.* should be easy)

In [2]:
# auxiliar code to visualize the computational graph
from utils import save_graph


In [2]:
# tensorflow 2.0 for loops
tf.keras.backend.clear_session()

## INITIALIZATION
matrix = np.zeros((5,8), dtype=np.int32)
matrix[1,2] = 1
matrix[2,3] = 1
print(matrix.shape)


def recurrent_step(_input, back_state, up_state, diagonal_state):
    # recurrent_states [BACK, UP, DIAGONAL]
    
    # new hidden state
    return back_state + up_state + diagonal_state + _input


# Base LOOP
@tf.function
def dynamic_tf_loop(input_data, recurrent_step):
    
    
    initial_state = tf.constant(0, dtype=tf.int32) # default state
    
    # shapes
    rows = input_data.shape[0]
    columns = input_data.shape[1]

    input_flat = tf.TensorArray(dtype=tf.int32, size=rows*columns)
    input_data = tf.reshape(input_data, [-1])
    input_flat = input_flat.unstack(input_data) # 1D data representation

    ######
    # find the recursive states base on the one dimentional index
    ######
    def get_back_state(index, states, columns):
        if 0>=index%columns: # out of the matrix/terminal case
            return initial_state
        else:
            return states.read(index-1)
 
    def get_up_state(index, states, columns):
        if index<columns: # out of the matrix/terminal case
            return initial_state
        else:
            return states.read(index-columns)

    def get_diagonal_state(index, states, columns):
        if 0>=index%columns or index<columns: # out of the matrix/terminal case
            return initial_state
        else:
            return states.read(index-columns-1)
    ######
    # END
    ######
    
    # flat matrix with all the hidden states
    # clear_after_read must be False, since some entries (e.g. diagonal) could read up on 3 times the same state value
    states = tf.TensorArray(dtype=tf.int32, size=rows*columns, clear_after_read = False)

    # sequential loop -> recursive loop
    for i in tf.range(rows*columns):
        states = states.write(i,
                              recurrent_step(input_flat.read(i),
                                             get_back_state(i, states, columns),
                                             get_up_state(i, states, columns),
                                             get_diagonal_state(i, states, columns))
                             )

        
    return states.stack() # tensor with all the hidden states

(5, 8)


In [4]:
all_h = save_graph(dynamic_tf_loop, input_data=matrix, recurrent_step=recurrent_step)

print(np.reshape(all_h, matrix.shape))

5 8
Tensor("TensorListLength:0", shape=(), dtype=int32)
[[  0   0   0   0   0   0   0   0]
 [  0   0   1   1   1   1   1   1]
 [  0   0   1   4   6   8  10  12]
 [  0   0   1   6  16  30  48  70]
 [  0   0   1   8  30  76 154 272]]


In [14]:
random_m = np.random.randint(0,3,size=(100,100), dtype=np.int32)

dynamic_tf_loop(random_m, recurrent_step)

<tf.Tensor: id=650, shape=(10000,), dtype=int32, numpy=
array([          0,           0,           2, ...,  -225304945,
          62681105, -1180517391], dtype=int32)>

In [13]:
# Simple example from TF docs: https://www.tensorflow.org/tutorials/customization/performance
# 

batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
    return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
    # [batch, time, features] -> [time, batch, features]
    input_data = tf.transpose(input_data, [1, 0, 2])
    max_seq_len = input_data.shape[0]

    states = tf.TensorArray(tf.float32, size=max_seq_len)
    state = initial_state
    
    for i in tf.range(max_seq_len):
        state = rnn_step(input_data[i], state)
        states = states.write(i, state)
    
    return tf.transpose(states.stack(), [1, 0, 2])
  
dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))


<tf.Tensor: id=648, shape=(2, 3, 4), dtype=float32, numpy=
array([[[0.06657267, 0.7333988 , 0.04778051, 0.36243272],
        [0.30615163, 1.0460746 , 0.68944657, 0.7993324 ],
        [0.6579858 , 1.8429313 , 0.939247  , 0.80793846]],

       [[0.68030334, 0.2816136 , 0.0358274 , 0.51169956],
        [1.0998355 , 0.5770551 , 0.5795438 , 1.3838853 ],
        [1.4530628 , 1.4685435 , 0.9450753 , 1.7760564 ]]], dtype=float32)>